# FPUS23 — YOLO Baseline (n/s/m) — v2

- Trains YOLO11 nano/small/medium on original FPUS23 YOLO dataset.
- Saves checkpoints every epoch to Drive and auto-resumes if rerun.
- Uses deterministic loaders and medical-friendly augment toggles.


In [ ]:
%pip -q install -U ultralytics >/dev/null
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
from pathlib import Path
YOLO_YAML = Path('/content/fpus23_project/dataset/fpus23_yolo/data.yaml')
if not YOLO_YAML.exists():
    maybe = Path('/content/fpus23/scripts/prepare_fpus23.py')
    if maybe.exists():
        !python /content/fpus23/scripts/prepare_fpus23.py --input /content/FPUS23_Dataset/Dataset --out_root /content/fpus23_project/dataset
assert YOLO_YAML.exists(), f'Missing {YOLO_YAML}. Prepare dataset first.'
print('Using', YOLO_YAML)


In [ ]:
# Train nano/small/medium. Per-epoch save + auto resume.
sizes = ['n','s','m']
batches = {'n':16,'s':16,'m':8}
for sz in sizes:
  run = f'fpus23_baseline_{sz}'
  !yolo train data=$YOLO_YAML model=yolo11{sz}.pt epochs=100 batch={batches[sz]} imgsz=768 \
       project='/content/drive/MyDrive/FPUS23_runs' name=$run deterministic=True workers=2 \
       cos_lr=True cls=3.0 resume=True save_period=1


In [ ]:
import pandas as pd
from pathlib import Path
BASE = Path('/content/drive/MyDrive/FPUS23_runs')
runs = ['fpus23_baseline_n','fpus23_baseline_s','fpus23_baseline_m']
rows = []
for r in runs:
  p = BASE / r / 'results.csv'
  if p.exists():
    df = pd.read_csv(p).dropna(how='all')
    if len(df):
      last = df.tail(1).squeeze()
      rows.append({'run': r, 'epoch': last.get('epoch'),
                   'mAP50': last.get('metrics/mAP50(B)'),
                   'mAP50-95': last.get('metrics/mAP50-95(B)'),
                   'precision': last.get('metrics/precision(B)'),
                   'recall': last.get('metrics/recall(B)'),
                   'csv': str(p)})
if rows:
  import pandas as pd
  display(pd.DataFrame(rows).sort_values('mAP50-95', ascending=False))
else:
  print('No results found yet. Check your Drive runs folder.')
